# Chapter 16: Coroutines

#### Disclaimer

This chapter is quite technical, and delves into themes that currently are beyond my understanding of computer science. Consequently, the notes from this chapter will be somewhat superficial.

This chapter deals with [coroutines](). A coroutine is syntactically like a generator: just a function with a `yield` keyword in its body.

However, in a coroutine, `yield` usually appears on the right side og an expression (e.g. `datum = yield`) , and it may or may not produce a value - if there is no value after the `yield` keyword, the generator yields `None`.

The **coroutine may receive data from the sender** using the `.send(datum)` method instead of `next(...)` to feed the coroutine.

Usually, the user pushes values to the coroutine.

Regardless of the flow of data, `yield` is a control flow device that can be used to implement cooperative multitasking: eache coroutine yields control to a central scheduler so that other coroutines can be activated.

## Digression: Coroutines explained to a five-year old

The [following explanation](https://www.reddit.com/r/learnprogramming/comments/29yzlm/eli5_coroutines/) is from the reddit user [zirak_](https://www.reddit.com/user/zirak_/):

(Note: This assumes you are a 5 year old programmer who knows words like "concurrency" and "asynchronous").

It's 13:00 o'clock in the afternoon, and you know what that means...it's time for Ms. Applebee, the most lovable kindergarten teacher in all of the kindergarten, to take us inside and play!

Accompanied by cries of joy and glee, you and several dozen other youngsters dashed to the playroom, where most of your toys were: Doll houses, their occupants, cars and pirates, board games and card games and a big ball pit.

You approached Ms. Applebee, who always smelled vaguely of peaches, whether you can play with the big Lego bricks. You always liked to build things like houses and evil lairs. "Why of course, snickerdoodle!" she replied, using your favourite nickname (a fact you deny to this day), "I'll go fetch the box of Legos!"

She came back with, as promised, a box filled with big plastic Lego pieces of different sizes and colours. But what's this? "Oh my, someone must have mixed all those different toys in one box! Never mind, I'm sure we can find a solution!"

"Now", she said, "you build your Lego masterpiece, and I can help you get all the Legos out of the box!"

But how exactly will that be done? Inspecting the universe, it looks like we have two functions:

1. `buildAwesomeLegoHouse`
2. `msApplebee`

How will `buildAwesomeLegoHouse` use `msApplebee`?

#### 1. Loops

Overconfident as you are, you say "Just weed out all the Legos and leave them to me!"

"As you wish", Ms. Applebee replied, and begins the tedious task of filtering out the Lego pieces. Some time passes. You stare intently at Ms. Applebee.

...

Kids around you play and frolic.

...

"There we go!" she exclaims once she's finally finished, "now play safe!", and throws the box of Legos at you. You now have to deal with both retrieving the next Lego piece, and doing something with it.

While it seems fine at first, you discover that it's a problem as the number of Lego bricks grow. Suddenly, you don't have to handle 10 bricks, you need to handle 100, and then 100,000! You can't possibly need all of those Lego bricks at once! What's more, it takes a while for Ms. Applebee to weed out all the yucky non-Legos. You just stood there like a fool, staring at her, doing nothing.

The above can be expressed in pseudo-Python as such:

```python
def buildAwesomeLegoHouse():
    legos = msApplebee()
    # TODO: Build house logic.

def msApplebee():
    return filter(isALegoBrick, boxOfToys)
```

This is clearly not ideal.

#### 2. Callbacks/Events

"How about whenever you find a Lego block, you tell me about it?" you suggest.

"Good idea! What a smart boy!" Ms. Applebee says and pats you on the head.

This goes better than the alternate universe, as two jobs can run in parallel (or seemingly in parallel, if only a single thread is present). You're given a block and you do something with it. Ms. Applebee gets a block, and gives it to you when you're free. If it's a multi-threaded environment, there's time for you and Ms. Applebee to do other things while you're waiting for the other to finish.

(Note: Under some implementations, instead of waiting for you to take the block each and every time, Ms. Applebee can fill a queue of blocks, from which you can get at your own pace. One notable example of this is the Windows Message Queue and, come to think of it, practically any socket library.)

In pseudo-Python (callback style):

```python
def buildAwesomeLegoHouse():
    def gotLego(lego):
        # TODO: Build house logic.
    msApplebee(gotLego)

def msApplebee(cb):
    for toy in boxOfToys:
        if isALegoBrick(toy):
            cb(toy)
```   

(Note that callbacks, event-listeners and Promises are identical in spirit, for better and for worse.)

This is all nice and dandy, and you manage to finish your Lego house. Awesome! But...what's Ms. Applebee doing? She keeps getting more and more blocks! "You can stop now, Ms. Applebee" you say, and she ignores. She keeps handing you blocks.

"Really, I'm all done!" More blocks.

"Shouldn't you be watching the other kids?" (you're a very responsible 5 year old). She keeps handing you blocks.

There's no true way in the callback model to signify "stop". With event listeners, you can stop paying attention to events, but Ms. Applebee will still be there, sorting out Lego bricks. Poor Ms. Applebee. There are some solutions to this problem, but none inherent in the model.

You can probably see where this is going.

#### 3. Co-routines

"How about I ask you for a block every time I need one?"

"Why, a great idea!"

Ms. Applebee gives you a block, and you happily arrange it. You then ask her nicely for another block, and then another, and so forth. When you're done with your awesome Lego house, you simply stop asking for more blocks.

In our beloved psuedo-Python:

```python
def buildAwesomeLegoHouse():
    for lego in msApplebee():
        # TODO: Build house logic.
        if houseIsFinished:
            break

def msApplebee():
    for toy in boxOfToys:
        if isALegoBrick(toy):
            yield toy
```

(Looks familiar?)

This combines the two previous methods. On the one hand, we have to wait for Ms. Applebee to filter out a Lego before we can put it. On the other, we don't have to deal with everything in one hit. Plus, we're once again the active party: We are only given blocks we asked for.

#### Conclusion

Co-routines are very much like callbacks (many things you can do with one, you can do with the other without working too much). However, they provide a cleaner interface over a function, give you greater control over the output, and enable you to feed data back into the co-routine. They are very powerful things when used correctly, and even if you don't see the need for them right now, remember they exist.

## Basic Behaviour of a Generator Used as a Coroutine

In [5]:
# Example 16-1. Simplest possible demonstration of a coroutine in action

def simple_coroutine():
    print("-> coroutine started")
    x = yield
    print("-> coroutine received: ", x)

In [6]:
coro = simple_coroutine()
coro

<generator object simple_coroutine at 0x10def0cd0>

In [7]:
next(coro)

-> coroutine started


In [8]:
coro.send(42)

-> coroutine received:  42


StopIteration: 

A coroutine can be in one of four states:

1. `GEN_CREATED`: Waiting to start exection.
2. `GEN_RUNNING`: Currently being executed.
3. `GEN_SUSPENDED`: Currently suspended at a `yield` expresison.
4. `GEN_CLOSED`: Execution completed.

Because the argument to the `send` method will become the value of the pending `yield` expression, you can only make a call like `coro.send(42)` *if the coroutine has been activated*.

The first activation of a coroutine is *always* done with `next(coro)`. This is called **priming** the coroutine. `coro.send(None)` has the same effect.

In [10]:
# Example 16-2. A coroutine that yields twice

from inspect import getgeneratorstate

def simple_coro2(a):
    print("-> Started: a = ", a)
    b = yield a
    print("-> Received: b = ", b)
    c = yield a + b
    print("-> Received c = ", c)

In [12]:
# Coroutine has not started
coro2 = simple_coro2(14)
getgeneratorstate(coro2)

'GEN_CREATED'

In [13]:
# Advance to first yield, yielding value of a
next(coro2)

-> Started: a =  14


14

In [14]:
# Coroutine suspended to wait for value to be assigned to b
getgeneratorstate(coro2)

'GEN_SUSPENDED'

In [15]:
# Send 28 to suspended coroutine
# yield evaluates to 28 and is bound to b
# The value of a + b is yielded (42)
# Coroutine is suspended waiting for the value to be assigned to c
coro2.send(28)

-> Received: b =  28


42

In [16]:
# Send number 99 to suspended coroutine
# yield evaluates to 99 and number is bound to c
# Coroutine terminates
coro2.send(99)

-> Received c =  99


StopIteration: 

In [17]:
# Coroutine execution has completed
getgeneratorstate(coro2)

'GEN_CLOSED'

**It is crucial to understand that the execution of a coroutine is suspended exactly at the `yield` keyword.**

In an assignment statement, the code to the right of `=` is evaluated *before* the actual assignment happens.

This means that in a line like `b = yield a`, the value of `b` will only be set when the coroutine is activated later by client code.

## Example: Coroutine to Compute a Running Average

The following example show how to implement **example 7-8** from [chapter 7]() as a coroutine:

In [28]:
# Example 16-3. code for running average coroutine

def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
        

In [29]:
avg = averager()
next(avg)

In [30]:
avg.send(10)

10.0

In [31]:
avg.send(5)

7.5

In [32]:
avg.send(25)

13.333333333333334

In [33]:
avg.close()
getgeneratorstate(avg)

'GEN_CLOSED'

Note the `.close()` must be called to terminate the coroutine. We can terminate coroutines in other ways. This will be covered in the **Coroutine Termination and Exception Handling** section below.

## Decorators for Coroutine Priming

It can be easy to forget to prime coroutines. Ramalho implements a decorator that serves this function:

In [34]:
# Example 16-5. decorator for priming a coroutine

from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer


In [39]:
# Example 16-6. decorated average coroutine

@coroutine
def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count


In [40]:
# We can immediately start send values to the coroutine
avg = averager()
avg.send(10)

10.0

In [41]:
avg.send(5)

7.5

In [42]:
avg.send(25)

13.333333333333334

In [43]:
avg.close()

## Note

We skip the following subchapters:

* **Coroutine Termination and Exception Handling** (pp. 488-491)
* **Returning a Value from a Coroutine** (pp. 491-493)

## Using `yield from`

Ramalho notes that `yield from` is an arguably misleading reuse of the keyword `yield`, and further notes that similar constructs in other languages are named `await`.

The key point is that when a generator `gen` calls `yield from subgen()`, the `subgen` will take over and will yield values to the caller of gen; the caller will in effect drive `subgen` directly.

Meanwhile, `gen` will be blocked until `subgen` terminates.

In [2]:
# Example: yield from as a shortcut to yield in a for loop

def gen():
    for c in "AB":
        yield c
    for i in range(1, 3):
        yield i

print(list(gen()))

def gen():
    yield from "AB"
    yield from range(1, 3)
    
print(list(gen()))

['A', 'B', 1, 2]
['A', 'B', 1, 2]


The main feature of `yield from` is to 

1. open a bidirectional channel from the outermost caller to the innermost subgenerator, so that values can ve sent and yielded back and forth directly from them, and 
2. exceptions can be thrown all the way in without adding a lot of exception handling boilerplate code in the intermediate coroutines.

The use of `yield from` requires some moving parts described in [PEP 380](https://www.python.org/dev/peps/pep-0380/):

*delegating generator*<br>
The generator function that contains the `yield from <iterable>` expression.

*subgenerator*<br>
The generator obtained from the `<iterable>` part of the `yield from` expression.

*caller*<br>
The client code that calls the delegating generator.

# Comments

Coroutines are complicated, and `yield from` is mind-bending. Return to this material and learn it properly *when you need it*. For now it's OK to be aware of the `yield` keyword, and the main idea of `yield from`.